<a href="https://colab.research.google.com/github/shriyakenkre/LondonBikeRides/blob/main/LondonBikeSharing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Installing the Kaggle Library
!pip install kaggle 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shriyakenkre","key":"6535aad2b9aaeb5c2ec4c8bd506e4b91"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
import pandas as pd
import zipfile
import kaggle

In [6]:
#download the dataset from kaggle using the kaggle API
!kaggle datasets download -d hmavrodiev/london-bike-sharing-dataset

  0% 0.00/165k [00:00<?, ?B/s]
100% 165k/165k [00:00<00:00, 81.7MB/s]


In [7]:
#Extract the file from the downloaded zip file 
zipfile_name = 'london-bike-sharing-dataset.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file:
  file.extractall()

In [8]:
#Read the csv file as a pandas dataframe
bikes = pd.read_csv("london_merged.csv")

In [9]:
#Explore the data 
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB


In [10]:
bikes.shape #To find the number of rows and columns

(17414, 10)

In [11]:
bikes

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...
17409,2017-01-03 19:00:00,1042,5.0,1.0,81.0,19.0,3.0,0.0,0.0,3.0
17410,2017-01-03 20:00:00,541,5.0,1.0,81.0,21.0,4.0,0.0,0.0,3.0
17411,2017-01-03 21:00:00,337,5.5,1.5,78.5,24.0,4.0,0.0,0.0,3.0
17412,2017-01-03 22:00:00,224,5.5,1.5,76.0,23.0,4.0,0.0,0.0,3.0


In [12]:
#countining the unique values in the weather code columns
bikes.weather_code.value_counts()

1.0     6150
2.0     4034
3.0     3551
7.0     2141
4.0     1464
26.0      60
10.0      14
Name: weather_code, dtype: int64

In [13]:
#count the unique values in the season column
bikes.season.value_counts()

0.0    4394
1.0    4387
3.0    4330
2.0    4303
Name: season, dtype: int64

In [14]:
#Specifying the column names that I want to use
New_Columns = {
    'timestamp' : 'time',
    'cnt' : 'count_of_bikes',
    't1' : 'Real_Temperature',
    't2' : 'Feels_Like_Temperature',
    'hum' : 'Humidity_Percentage',
    'wind_speed' : 'Wind_Speed_kph',
    'weather_code' : 'Weather',
    'is_holiday' : 'is_holiday',
    'is_weekend' : 'is_weekend',
    'season' : 'Season'
}

In [15]:
#Renaming the columns to the specified column names 
bikes.rename(New_Columns,axis=1,inplace = True)

In [16]:
#Changing the humidity values from percentage to decimals 
bikes.Humidity_Percentage = bikes.Humidity_Percentage/100

In [17]:
#Creating a dictionary for Seasons so that we can actually map the integers to the actual values 
Season_Dict = {
    '0.0' : 'Spring',
    '1.0' : 'Summer',
    '2.0' : 'Autumn',
    '3.0' : 'Winter'
}

#Creating a dictionary for Weather so that we can actually map the integeres to the actual values
Weather_Dict = {
    '1.0' : 'Clear',
    '2.0' : 'Scattered Clouds',
    '3.0' : 'Broken Clouds',
    '4.0' : 'Cloudy',
    '7.0' : 'Rain',
    '10.0' : 'Rain with thunderstorm',
    '26.0' : 'Snowfall'
}



In [18]:
#Changing the seasons column data type to string 
bikes.Season = bikes.Season.astype('str')
#Mapping the integer values to the actual season
bikes.Season = bikes.Season.map(Season_Dict)

#Changing the weather column data type to string 
bikes.Weather = bikes.Weather.astype('str')
#Mapping the integer values to the actual Weather
bikes.Weather = bikes.Weather.map(Weather_Dict)

In [19]:
#Checking our dataframe to see if the mappings have worked
bikes.head()

,time,count_of_bikes,Real_Temperature,Feels_Like_Temperature,Humidity_Percentage,Wind_Speed_kph,Weather,is_holiday,is_weekend,Season
0,2015-01-04 00:00:00,182,3.0,2.0,0.930,6.0,Broken Clouds,0.0,1.0,Winter
1,2015-01-04 01:00:00,138,3.0,2.5,0.930,5.0,Clear,0.0,1.0,Winter
2,2015-01-04 02:00:00,134,2.5,2.5,0.965,0.0,Clear,0.0,1.0,Winter
3,2015-01-04 03:00:00,72,2.0,2.0,1.000,0.0,Clear,0.0,1.0,Winter
4,2015-01-04 04:00:00,47,2.0,0.0,0.930,6.5,Clear,0.0,1.0,Winter


In [20]:
#Writing the final dataframe to an excel file that we will use in our data visualization
bikes.to_excel('london_bikes_final.xlsx', sheet_name = 'Data')